In [1]:
# imports and settings
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm
import cv2
import glob
import numpy.typing as npt
from math import log10, floor

pd.options.plotting.backend = "plotly"

2022-11-04 18:13:13.418776: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-04 18:13:13.542694: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# import and prepare data
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
images = np.append(train_images, test_images, axis=0)
images = images.reshape(images.shape[0], 28, 28, 1).astype('float32')

In [52]:
# run dct on images and gather first significant digits
fsd = []
#for image in tqdm(images):
dcts = np.array([cv2.dct(image) for image in images[798:799]])
print(f"dct shape: {dcts.shape}")
dcts = dcts.flatten()
print(f"flattened shape: {dcts.shape}")
dcts = dcts[dcts != 0]
for n in tqdm(dcts):
    #for n in dct:
    num = int(abs(n * (10 ** -int(floor(log10(abs(n))))))) if n != 0 else 0
    fsd.append(num)
print(f"Len: {len(fsd)}")
fsd = np.array(fsd)
print(np.sum(fsd))
print(fsd)

dct shape: (1, 28, 28)
flattened shape: (784,)


100%|██████████| 784/784 [00:00<00:00, 176498.01it/s]

Len: 784
2591
[6 2 7 1 2 3 7 2 2 6 4 8 5 1 9 3 6 3 3 1 1 2 9 2 1 1 1 1 1 1 1 1 2 1 2 1 1
 1 1 9 6 6 3 3 4 4 3 3 2 4 1 2 3 4 4 4 3 6 2 2 1 3 2 1 3 8 2 1 9 1 1 5 7 1
 1 1 2 6 1 3 1 2 2 4 2 1 1 2 8 6 3 7 3 1 4 5 9 5 9 1 2 4 6 1 9 2 8 9 9 8 1
 1 2 1 2 2 2 1 2 6 2 1 8 1 9 1 5 1 4 2 7 1 2 8 4 9 1 4 1 2 1 4 1 1 1 7 5 7
 1 6 1 1 1 3 1 4 4 4 8 7 6 3 4 2 2 1 2 7 2 1 1 1 2 8 7 1 1 1 1 9 1 2 1 4 7
 2 1 2 2 1 2 3 3 2 1 6 2 9 2 1 1 8 8 1 5 3 2 2 1 2 1 4 2 3 3 1 3 8 8 8 9 1
 9 2 7 3 3 2 6 1 6 1 8 4 5 1 3 3 3 4 9 1 2 3 7 1 2 6 2 2 1 4 1 1 9 1 6 4 1
 5 8 1 3 6 2 1 3 3 7 1 1 1 2 1 1 3 1 1 1 1 2 5 2 1 1 4 6 3 4 6 3 7 1 4 1 2
 2 8 1 2 5 4 1 7 1 2 9 3 3 1 3 5 7 1 8 8 2 4 2 1 1 1 4 2 1 3 2 1 5 1 1 5 2
 1 2 3 9 7 1 6 9 6 4 5 1 7 4 8 2 1 1 2 1 5 1 5 1 2 6 1 4 1 1 1 1 4 1 8 9 4
 5 2 4 6 2 4 7 1 9 4 7 1 1 3 2 2 2 1 3 3 2 5 3 1 5 4 8 1 7 5 1 1 1 1 2 1 1
 7 6 3 7 2 4 7 1 8 1 2 8 2 2 5 2 4 7 5 9 5 5 2 2 1 2 1 2 2 1 1 4 2 2 4 1 9
 1 1 1 6 4 1 5 3 2 2 4 2 9 4 2 2 1 4 2 4 1 9 7 9 1 7 9 7 3 1 4 1 1 2 6 9 6
 6 7 3 1 8 

In [55]:
# NUMPY run dct on images and gather first significant digits
fsd_fast = [] #np.array([]).astype(int)

dcts = np.array([cv2.dct(image) for image in images[798:799]])
print(f"dct shape: {dcts.shape}")
#for image in tqdm(images):
    #for n in dct:
    #n = cv2.dct(image).flatten()
n = dcts.flatten()
print(f"flattened shape: {n.shape}")
n = n[n != 0]
print(n)
# TODO: Find out why image 798 at i=14 has a fsd zero with this method
n = np.abs(n * np.power(np.full(n.shape, 10.), -np.floor(np.log10(np.abs(n))).astype(int))).astype(int)
fsd_fast.extend(list(n))
# print(fsd)
print(f"Len: {len(fsd_fast)}")
fsd_fast = np.array(fsd_fast)
s = np.sum(fsd_fast)
print(s)
print(fsd_fast)

print(np.sum(fsd) == np.sum(fsd_fast))

dct shape: (1, 28, 28)
flattened shape: (784,)
[ 6.98500000e+02 -2.27511768e+01 -7.18363525e+02  1.96880081e+02
  2.83554565e+02 -3.73615753e+02 -7.15634689e+01  2.66423492e+02
 -2.32996559e+01 -6.97395096e+01  4.13192635e+01  8.69359821e-02
  5.02723579e+01  1.38448391e+01 -9.99999847e+01  3.09173369e+00
  6.53423157e+01 -3.71951981e+01 -3.53410492e+01  1.61504364e+01
  1.31802130e+01  2.37057705e+01 -9.69152153e-01 -2.72459583e+01
  1.49260960e+01  1.71102676e+01 -1.97412663e+01 -1.09282560e+01
 -1.81070595e+01  1.04033371e+02  1.39073196e+02 -1.70442154e+02
 -2.66389191e+02  1.05304489e+02  2.29067322e+02 -1.10452225e+02
 -1.67654388e+02  1.30175446e+02  1.30932327e+02 -9.01856918e+01
 -6.88835678e+01  6.01914597e+01  3.76000862e+01 -3.80074234e+01
 -4.25695572e+01  4.69331837e+00  3.15990639e+01  3.70132232e+00
 -2.04186344e+01 -4.74088526e+00  1.62953453e+01  2.37587986e+01
 -3.12814736e+00 -4.09474068e+01 -4.80914068e+00  4.76850853e+01
 -3.31506134e+02  6.09326935e+01  2.4417218

In [54]:
for i in range(len(fsd)):
    if fsd[i] != fsd_fast[i]:
        print(f"different at index {i}, {fsd[i]} != {fsd_fast[i]}")

different at index 14, 9 != 0


In [17]:
# count fds
count = []
for i in range(1,10):
    count.append(np.count_nonzero(fsd == i))
# count = count / np.sum(count)

count_fast = []
for i in range(1,10):
    count_fast.append(np.count_nonzero(fsd_fast == i))
# count_fast = count_fast / np.sum(count_fast)

print(count)
print(count_fast)

[17772104, 10255017, 6621158, 4830037, 3889101, 3337822, 2972362, 2706098, 2496300]
[17772104, 10255017, 6621158, 4830037, 3889101, 3337822, 2972362, 2706098, 2496284]


In [ ]:
# generate ground truth benfords law
bf_law = []
for i in range(1,10):
    bf_law.append(log10(1 + (1 / i)))

In [ ]:
# plot data tp compare fsd vs benfords law
df = pd.DataFrame()
df["digit"] = [1,2,3,4,5,6,7,8,9]
df["MNIST FSD count"] = count
df["Benfords Law (ground truth)"] = bf_law

df.plot(x="digit", y=["MNIST FSD count", "Benfords Law (ground truth)"],
        labels={
            "digit" : "First Significant Digit (FSD)",
            "value" : "Probability"
        })

In [ ]:
horses = np.array([cv2.imread(file, cv2.IMREAD_GRAYSCALE) for file in glob.glob("horses/000000/*.png")]).astype("float32")
print(horses.shape)

fsd = []
for horse in tqdm(horses):
    for dct in cv2.dct(horse):
        for n in dct:
            num = int(abs(n * (10 ** -int(floor(log10(abs(n))))))) if n != 0 else 0
            fsd.append(num)
fsd = np.array(fsd)

# count fds
count = []
for i in range(1,10):
    count.append(np.count_nonzero(fsd == i))
count = count / np.sum(count)
print(count)

bf_law = []
for i in range(1,10):
    bf_law.append(log10(1 + (1 / i)))

df = pd.DataFrame()
df["digit"] = [1,2,3,4,5,6,7,8,9]
df["MNIST FSD count"] = count
df["Benfords Law (ground truth)"] = bf_law

df.plot(x="digit", y=["MNIST FSD count", "Benfords Law (ground truth)"],
        labels={
            "digit" : "First Significant Digit (FSD)",
            "value" : "Probability"
        })
